<a href="https://colab.research.google.com/github/baoquan25/scrape_data/blob/main/Auto_scraping_Linkedin_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.common.by import By
import csv
from bs4 import BeautifulSoup

In [ ]:
driver = webdriver.Chrome()
url ='https://www.linkedin.com/login'
driver.get(url)
sleep(2)
print('-finish initializing a driver')

# Đọc file
credential = open('F:/login_credential.txt')
username = credential.readline().strip()
password = credential.readline().strip()
print('-finish import the login credential')

# Nhập email
email_field = driver.find_element(By.ID, 'username')
email_field.send_keys(username)
sleep(3)
print('-finish key in email')

# Nhập password
password_field = driver.find_element(By.NAME, 'session_password')
password_field.send_keys(password)
sleep(2)
print('-finish key in password')

# CLick vào nút đăng nhập
login_button = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[4]/button')
login_button.click()
print('- Finish logging in')

In [ ]:
search_field = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')

# Input the search query to the search bar
search_query = input('what profile do you want to scrape')
search_field.send_keys(search_query)

# Search
search_field.send_keys(Keys.RETURN)
print('- Finish searching')

In [ ]:
# lấy 1 trang đầu
def GetURL():
    # Lấy mã nguồn HTML của trang hiện tại
    page_source = BeautifulSoup(driver.page_source, "html.parser")

    # Tìm tất cả các thẻ <a> có class "app-aware-link" và href chứa "/in/"
    profiles = page_source.find_all('a', class_='ITnlIYRaRoJycfooUBNruLtOOXZuqVGU',target=lambda t: t != '_self')

    # Tạo danh sách để lưu các URL hồ sơ
    all_profile_URL = []

    for profile in profiles:
        profile_URL = profile.get('href')  # Lấy giá trị của href
        # Kiểm tra nếu href chứa "/in/" (chỉ là các hồ sơ cá nhân)
        if profile_URL and "/in/" in profile_URL:
            if profile_URL not in all_profile_URL:  # Tránh trùng lặp
                all_profile_URL.append(profile_URL)

    return all_profile_URL

# Gọi hàm và in kết quả
print(GetURL())

In [ ]:
def GetURLsonPages():
    input_page = int(input('How many pages you want scrape'))
    URLs_all_page = []
    for page in range(input_page):
        URLs_one_page = GetURL()
        sleep(2)

        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        sleep(2)

        next_button = driver.find_element(By.CLASS_NAME, 'artdeco-pagination__button--next')
        next_button.click()
        sleep(2)

        URLs_all_page.extend(URLs_one_page)
    return URLs_all_page


# Lấy danh sách URL
URLs_all_page = GetURLsonPages()

# Mở file CSV để ghi dữ liệu
with open('output.csv', 'w', newline='', encoding='utf-8-sig') as file_output:
    headers = ['Name', 'Job title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n', fieldnames=headers)
    writer.writeheader()

    # Lặp qua tất cả các URL và lấy thông tin
    for idx, linkedin_URL in enumerate(URLs_all_page, start=1):

        # Mở URL trên trình duyệt
        driver.get(linkedin_URL)

        # Lấy mã nguồn HTML của trang
        page_source = BeautifulSoup(driver.page_source, 'html.parser')


        info_div = page_source.find('div', class_ ='mt2 relative')
        if not info_div:
            continue  # Bỏ qua URL này và chuyển sang URL tiếp theo


        # Tìm thẻ h1 có tên nhân viên
        name = info_div.find('h1').get_text().strip()
        print('tên nhân viên: ',name)

        # tìm nghề nghiệp
        job = info_div.find('div', class_='text-body-medium break-words').get_text().strip()
        print('nghề nghiệp: ',job)

        #tìm vị trí
        location = info_div.find('span', class_='text-body-small inline t-black--light break-words').get_text().strip()
        print('vị trí: ',location)

        #in đường link
        print('URL :',linkedin_URL)
        print('-' * 50)

        writer.writerow({'Name': name, 'Job title': job, 'Location': location, 'URL': linkedin_URL})

print("Dữ liệu đã được ghi vào output.csv.")